In [21]:
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
random_seed = 42
learning_rate = 0.001
batch_size = 32
n_epochs = 15

img_size = 32
n_classes = 10

In [6]:
def get_accuracy(model, data_loader, device):
    
    correct_pred = 0
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:
            X , y_true = X.to(device), y_true.to(device)
            
            _, y_pred = model(X)
            _, predicted_labels = torch.max(y_pred, 1)
            
            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()
            
    return correct_pred.float() / n

In [7]:
def plot_losses(train_losses, valid_losses):
    plt.style.use('seaborn')
    
    train_losses = np.array(train_losses)
    valid_losses = np.array(valid_losses)
    
    fig, ax = plt.subplots(figsize = (8, 4.5))
    ax.plot(train_losses, color = 'blue', label = 'Training Loss')
    ax.plot(valid_losses, color = 'red', label = 'validation Loss')
    ax.set(title = "Loss over epochs",
          xlabel = "Epoch",
          ylabel = "Loss")
    ax.legend()
    fig.show()
    
    plt.style.use('default')

In [9]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for X, y_true in train_loader:
        optimizer.zero_grad()
        
        X, y_true = X.to(device), y_true.to(device)
        y_hat, _ =  model(X)
        loss = criterion(y_hat, y_true)
        running_loss += loss.item() * X.size(0)
        
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

In [19]:
def validation(valid_loader, model, criterion, device):
    model.eval()
    running_loss = 0
    for X, y_true in valid_loader:
        X, y_true = X.to(device), y_true.to(device)
        
        y_hat, _ = model(X)
        loss = criterion(y_hat, y_true)
        running_loss += loss.item() * X.size(0)
    
    epoch_loss = running_loss / len(valid_loader.dataset)
    
    return model, epoch_loss

In [17]:
def training_loop(model,criterion, optimizer, train_loader, valid_loader, epochs, device, print_every = 1):
    best_loss = 1e10
    train_losses = []
    valid_losses = []
    
    for epoch in range(epochs):
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)
        
        with torch.no_grad():
            model, valid_loss = validation(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)
            
        if epoch % print_every == (print_every - 1):
            train_acc = get_accuracy(model, train_loader, device = device)
            valid_acc = get_accuracy(model, valid_loader, device = device)
            
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')
            
    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

In [12]:
transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])

train_dataset = datasets.MNIST(root = 'mnist_data',
                              train = True,
                              transform= transform,
                              download = True)

valid_dataset = datasets.MNIST(root = 'mnist_data',
                              train = False,
                              transform = transform,
                              download = True)

train_loader = DataLoader(dataset = train_dataset,
                         batch_size = batch_size,
                         shuffle = True)

valid_loader = DataLoader(dataset = valid_dataset,
                         batch_size = batch_size,
                         shuffle = False)

In [13]:
class LeNet5(nn.Module):
    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5, stride = 1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size = 2),
            nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5, stride = 1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size = 2),
            nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = 5, stride = 1),
            nn.Tanh()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features = 120, out_features = 84),
            nn.Tanh(),
            nn.Linear(in_features = 84, out_features= n_classes),
        )
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim = 1)
        return logits, probs

In [14]:
model = LeNet5(n_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()

In [22]:
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, n_epochs, device)